In [3]:
!pip install -q kaggle
! mkdir /root/.kaggle
! cp /content/kaggle.json /root/.kaggle
! chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d shubhammeshram579/bank-customer-churn-prediction

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  0% 0.00/262k [00:00<?, ?B/s]
100% 262k/262k [00:00<00:00, 112MB/s]


In [4]:
!unzip /content/bank-customer-churn-prediction.zip

Archive:  /content/bank-customer-churn-prediction.zip
  inflating: Churn_Modelling.csv     


In [5]:
import pandas as pd
import numpy as np
import torch
import plotly.express as px
import torch.nn as nn

In [6]:
### Logistic Regression : Sigmoid Regression ??(Binary Classification) + SoftMax Regression ( Multi Class Classification)

In [7]:
# ### Ingredients Required :
# 1. Dataset == > cleaned + proecessed + encoded + scaled --> into Tensor Dataset ==> Tensor Data Loader with Batches

# 2. Model / Network : Feed Forward Network  : Multi Layer Perceptron - > Different Number of Layers , Batch sizes , learning rate and Optimizer , Activation , Drop Out

# 3. Error Function

# 4. Entire Loop

# 5. Evaluation of Classification Outputs

In [8]:
churn_df = pd.read_csv("/content/Churn_Modelling.csv")

In [9]:
churn_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,3,15619304,Onio,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,4,15701354,Boni,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43.0,2,125510.82,1,NaN,1.0,79084.10,0


In [10]:
churn_df = churn_df.drop_duplicates(subset=["CustomerId"]).drop("RowNumber",axis=1).set_index(["CustomerId","Surname"])

In [11]:
churn_df = churn_df.dropna()

In [12]:
px.box(churn_df , x="Exited",y="CreditScore")

In [13]:
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9996 entries, (15634602, 'Hargrave') to (15628319, 'Walker')
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      9996 non-null   int64  
 1   Geography        9996 non-null   object 
 2   Gender           9996 non-null   object 
 3   Age              9996 non-null   float64
 4   Tenure           9996 non-null   int64  
 5   Balance          9996 non-null   float64
 6   NumOfProducts    9996 non-null   int64  
 7   HasCrCard        9996 non-null   float64
 8   IsActiveMember   9996 non-null   float64
 9   EstimatedSalary  9996 non-null   float64
 10  Exited           9996 non-null   int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 1.3+ MB


In [14]:
X_df = churn_df.drop("Exited",axis=1)
y_df = churn_df[["Exited"]]

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
### Label encode the Geography

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X_df["Gender"] = le.fit_transform(X_df["Gender"])


le = LabelEncoder()
X_df["Geography"] = le.fit_transform(X_df["Geography"])



In [17]:
X_train , X_test , Y_train , Y_test = train_test_split(X_df, y_df , test_size=0.2 , random_state = 99)

In [19]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

In [20]:
from torch.utils.data import TensorDataset , DataLoader

In [21]:
X_train_scaled = torch.Tensor(X_train_scaled)
Y_train = torch.Tensor(Y_train["Exited"].values)

In [22]:
X_test_scaled = torch.Tensor(X_test_scaled)
Y_test = torch.Tensor(Y_test["Exited"].values)

In [23]:
train_ds = TensorDataset(X_train_scaled,Y_train)
test_ds = TensorDataset(X_test_scaled,Y_test)


In [79]:
train_loader = DataLoader(train_ds, batch_size = 128 ,shuffle = True)
test_loader = DataLoader(test_ds, batch_size = 32 ,shuffle = True)

In [80]:
X_test_scaled.shape

torch.Size([2000, 10])

In [81]:
[16,32,16]

[16, 32, 16]

In [110]:
### Model


class LogisticRegression(nn.Module):
  def __init__(self, input_size, hidden_layer:list, output_size):
    super(LogisticRegression, self).__init__()

    layers = []

    layers.append(nn.Linear(input_size,hidden_layer[0]))
    layers.append(nn.ReLU())

    for layer in range(1,len(hidden_layer)):
      layers.append(nn.Linear(hidden_layer[layer-1],hidden_layer[layer]))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(0.3))


    layers.append(nn.Linear(hidden_layer[-1],output_size))
    layers.append(nn.Dropout(0.2))
    layers.append(nn.Sigmoid())

    self.layers = nn.Sequential(*layers)

  def forward(self, x):
    return self.layers(x)

In [140]:
model = LogisticRegression(10,[15,10,5],1)

In [141]:
model

LogisticRegression(
  (layers): Sequential(
    (0): Linear(in_features=10, out_features=15, bias=True)
    (1): ReLU()
    (2): Linear(in_features=15, out_features=10, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=10, out_features=5, bias=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=5, out_features=1, bias=True)
    (9): Dropout(p=0.2, inplace=False)
    (10): Sigmoid()
  )
)

In [142]:
for name ,layer in model.named_children():
  print(name)
  print(name, sum([p.numel() for p in layer.parameters()]))

layers
layers 386


In [151]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.9,step_size=5)


In [144]:
from sklearn.metrics import accuracy_score , classification_report

In [145]:
from tqdm import tqdm

In [152]:
num_epochs = 300
train_loss = []
test_loss = []
accuracy_oa = []

for epoch in tqdm(range(num_epochs)):

  batch_test_loss = []
  batch_train_loss = []
  accuracy = []



  model.train()

  for inputs , target in train_loader:
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs,target.view(-1,1))
      loss.backward()
      optimizer.step()
      batch_train_loss.append(loss.item())
      scheduler.step()

  model.eval()
  with torch.no_grad():
     for inputs , target in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs,target.view(-1,1))
        batch_test_loss.append(loss.item())
        predicted = (outputs.detach().numpy()>0.5).astype(int)
        accuracy.append(accuracy_score(target,predicted))

  train_loss.append(np.mean(batch_train_loss))
  test_loss.append(np.mean(batch_test_loss))
  accuracy_oa.append(np.mean(accuracy))





100%|██████████| 300/300 [01:11<00:00,  4.22it/s]


In [153]:
px.line(y=[train_loss,test_loss])

In [147]:
px.line(y=[train_loss,test_loss])

In [139]:
px.line(y=[train_loss,test_loss])

In [148]:
px.line(y=[accuracy_oa])